In [21]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from os import makedirs, getcwd

# from pyfinance.ols import PandasRollingOLS
# replaces pyfinance.ols.PandasRollingOLS (no longer maintained)


import yfinance as yf
# from yahoofinancials import YahooFinancials

In [6]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "XLK XLP XLB XLF XLV XLY XLE XLU XLI",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "max",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = False,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = False,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )



[*********************100%***********************]  9 of 9 completed


In [21]:
from pandas import read_csv, to_datetime, MultiIndex, merge, concat
from functools import reduce

In [19]:
data_path = '/Users/yueyuchen/Documents/Academy/Research/MarketWatch/data/daily'

tickers = "XLK XLP XLB XLF XLV XLY XLE XLU XLI"

dfs = []

for tic in tickers.split(' '):

    df = read_csv(f'{data_path}/{tic}.csv')
    df['Date'] = to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df.columns = MultiIndex.from_product([[tic], df.columns])
    dfs.append(df)

data = reduce(lambda x, y: merge(left=x, right=y, left_index=True, right_index=True, how='inner'), dfs)



In [20]:
data.head()

XLK                       ...       XLI                  
                Open       High       Low  ...     Close  Adj Close Volume
Date                                       ...                            
1998-12-22  32.40625  32.500000  31.78125  ...  23.28125  15.299635    600
1998-12-23  32.75000  32.812500  32.15625  ...  23.68750  15.566610    900
1998-12-24  32.81250  32.812500  32.56250  ...  24.00000  15.771975   1400
1998-12-28  33.18750  33.187500  32.59375  ...  24.12500  15.854114   7400
1998-12-29  32.93750  33.015625  32.50000  ...  24.46875  16.080025   1400

[5 rows x 54 columns]

In [27]:
sec_data = data
sec_data.index = sec_data.index.rename('date')
for tick in sec_data.columns.get_level_values(0).unique():
    
    sec_data[tick].rename(columns={x:x.lower() for x in sec_data[tick].columns}).to_csv(f'data/{tick}.csv')

In [28]:
sec_data = []
for t in data.columns.get_level_values(0).unique():
    
    sec_data.append(data[t].assign(ticker=t))
    
sec_data = concat(sec_data, axis=0)
sec_data = sec_data.set_index('ticker', append=True)
sec_data = sec_data.swaplevel()
sec_data.to_csv('data/raw_sec_data.csv')
sec_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 53874 entries, ('XLK', Timestamp('1998-12-22 00:00:00')) to ('XLI', Timestamp('2022-10-05 00:00:00'))
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       53874 non-null  float64
 1   High       53874 non-null  float64
 2   Low        53874 non-null  float64
 3   Close      53874 non-null  float64
 4   Adj Close  53874 non-null  float64
 5   Volume     53874 non-null  int64  
dtypes: float64(5), int64(1)
memory usage: 2.8+ MB


In [29]:
sec_data

Open       High  ...  Adj Close   Volume
ticker date                              ...                    
XLK    1998-12-22  32.406250  32.500000  ...  24.530024   300500
       1998-12-23  32.750000  32.812500  ...  25.116072   338300
       1998-12-24  32.812500  32.812500  ...  25.020390   243700
       1998-12-28  33.187500  33.187500  ...  25.092144   309300
       1998-12-29  32.937500  33.015625  ...  25.163906   123800
...                      ...        ...  ...        ...      ...
XLI    2022-09-29  84.669998  84.750000  ...  83.940002  9332455
       2022-09-30  83.699997  84.440002  ...  82.839996  8935651
       2022-10-03  83.760002  86.010002  ...  85.339996  8612500
       2022-10-04  86.639999  88.330002  ...  88.309998  9574493
       2022-10-05  87.349998  88.580002  ...  87.830002  7055800

[53874 rows x 6 columns]